In [2]:
import os
import logging
import numpy as np
import pydicom
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import matplotlib.pyplot as plt
import random

# Configuration des logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# ====================
# PARAMÈTRES GÉNÉRAUX
# ====================
IMG_SIZE = 224
BATCH_SIZE = 32

# Nombre d’époques
EPOCHS_INITIAL = 30  # Phase 1
EPOCHS_FINE = 10     # Phase 2

LEARNING_RATE_INITIAL = 1e-4
LEARNING_RATE_FINE = 1e-5

# Chemin vers le fichier pickle
NOTEBOOK_DIRECTORY = "/Users/dtilm/Desktop/P1-Classification/notebook/balanced_data.pkl"


# ====================
# CHARGEMENT DES DONNÉES
# ====================
logger.info("Chargement des métadonnées fusionnées depuis balanced_data.pkl")
balanced_data = pd.read_pickle(NOTEBOOK_DIRECTORY)

# Filtrage des données pour inclure uniquement les colonnes requises et non nulles
logger.info("Filtrage des données pour inclure uniquement les colonnes requises et non nulles.")
filtered_data = balanced_data[
    [
        'image_file_path_dicom',
        'pathology',
        'abnormality_type',
        'left_or_right_breast',
        'image_view'
    ]
].dropna()

label_mapping = {'BENIGN_WITHOUT_CALLBACK': 0, 'BENIGN': 1, 'MALIGNANT': 2}
filtered_data['label'] = filtered_data['pathology'].map(label_mapping)

logger.info(f"Nombre total d'images après filtrage : {len(filtered_data)}")
print(filtered_data['label'].value_counts())

logger.info("Division des données en ensembles d'entraînement et de validation.")
train_data, val_data = train_test_split(
    filtered_data, 
    test_size=0.2, 
    random_state=42, 
    stratify=filtered_data['label']
)

train_paths = train_data['image_file_path_dicom'].tolist()
train_labels = train_data['label'].tolist()
val_paths = val_data['image_file_path_dicom'].tolist()
val_labels = val_data['label'].tolist()

logger.info(f"Nombre d'images pour l'entraînement : {len(train_paths)}")
logger.info(f"Nombre d'images pour la validation : {len(val_paths)}")


# ====================
# DATA AUGMENTATION
# ====================
data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1, 0.1),
])

# ====================
# FONCTION DE PRÉTRAITEMENT
# ====================
def preprocess_dicom(file_path):
    try:
        dicom = pydicom.dcmread(file_path, force=True)
        img = dicom.pixel_array.astype(np.float32)

        img -= np.min(img)
        max_val = np.max(img)
        if max_val == 0:
            return None
        img = (img / max_val) * 255.0

        img = np.stack([img, img, img], axis=-1)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = preprocess_input(img)

        return img

    except Exception as e:
        logger.error(f"Erreur DICOM {file_path}: {e}")
        return None

# ====================
# EXEMPLE VISUEL (OPTIONNEL)
# ====================
def show_random_preprocessed_images(paths, num_images=5):
    plt.figure(figsize=(15, 8))
    for i in range(num_images):
        path = random.choice(paths)
        preprocessed_img = preprocess_dicom(path)
        if preprocessed_img is not None:
            preprocessed_img_np = preprocessed_img.numpy()
            min_val = preprocessed_img_np.min()
            max_val = preprocessed_img_np.max()
            if max_val > min_val:
                disp_img = (preprocessed_img_np - min_val) / (max_val - min_val)
            else:
                disp_img = preprocessed_img_np
            plt.subplot(1, num_images, i + 1)
            plt.imshow(disp_img)
            plt.title(f"Sample {i+1}")
            plt.axis('off')
    plt.tight_layout()
    plt.show()

# show_random_preprocessed_images(train_paths, num_images=5)

# ====================
# GÉNÉRATEURS DE DONNÉES
# ====================
class DICOMDataGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, augment=False, shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(len(self.file_paths))
            np.random.shuffle(indices)
            self.file_paths = [self.file_paths[i] for i in indices]
            self.labels = [self.labels[i] for i in indices]

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.file_paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        logger.info(f"Traitement du batch {idx + 1}/{self.__len__()}: {len(batch_paths)} images.")

        images = []
        valid_labels = []

        for path, lbl in zip(batch_paths, batch_labels):
            img = preprocess_dicom(path)
            if img is not None:
                if self.augment:
                    img = tf.expand_dims(img, 0)
                    img = data_augmentation(img)
                    img = tf.squeeze(img, axis=0)
                images.append(img)
                valid_labels.append(lbl)

        images = np.array(images)
        labels_array = np.array(valid_labels)

        logger.info(f"Batch {idx + 1} chargé : {len(images)} images valides.")
        return images, tf.keras.utils.to_categorical(labels_array, num_classes=3)

gen_train = DICOMDataGenerator(train_paths, train_labels, BATCH_SIZE, augment=True, shuffle=True)
gen_val = DICOMDataGenerator(val_paths, val_labels, BATCH_SIZE, augment=False, shuffle=False)

# ====================
# CONSTRUCTION DU MODÈLE
# ====================
logger.info("Définition du modèle EfficientNetB0.")

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Phase 1 : on gèle le backbone
base_model.trainable = False

# --- AMÉLIORATION : COUCHE DENSE CACHÉE ---
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    # nouvelle couche dense
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # sortie
])

# Compilation initiale (phase 1)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_INITIAL),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ====================
# CALLBACKS
# ====================
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
]

# ====================  
# ENTRAÎNEMENT - PHASE 1
# ====================
logger.info("Début de l'entraînement du modèle (phase 1).")

history = model.fit(
    gen_train,
    validation_data=gen_val,
    epochs=EPOCHS_INITIAL,  # 30
    callbacks=callbacks,
    verbose=1
)

logger.info("Phase 1 d'entraînement terminée. Début du fine-tuning.")

# ====================
# FINE-TUNING - PHASE 2
# ====================
nb_layers = len(base_model.layers)
# On dé-gèle 50 % des couches finales du backbone
for layer in base_model.layers[-int(nb_layers * 0.5):]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    gen_train,
    validation_data=gen_val,
    epochs=EPOCHS_FINE,  # 10
    callbacks=callbacks,
    verbose=1
)

logger.info("Fine-tuning terminé. Sauvegarde du modèle.")

model.save('efficientnet_b0_updated_multiclass.h5')
logger.info("Modèle sauvegardé avec succès.")


2025-01-29 15:01:57,392 - Chargement des métadonnées fusionnées depuis balanced_data.pkl
2025-01-29 15:01:57,398 - Filtrage des données pour inclure uniquement les colonnes requises et non nulles.
2025-01-29 15:01:57,406 - Nombre total d'images après filtrage : 948
2025-01-29 15:01:57,409 - Division des données en ensembles d'entraînement et de validation.
2025-01-29 15:01:57,415 - Nombre d'images pour l'entraînement : 758
2025-01-29 15:01:57,415 - Nombre d'images pour la validation : 190
2025-01-29 15:01:57,468 - Définition du modèle EfficientNetB0.


label
1    316
2    316
0    316
Name: count, dtype: int64


2025-01-29 15:01:58,406 - At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.
2025-01-29 15:01:58,408 - There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `tf.keras.optimizers.legacy.Adam`.
2025-01-29 15:01:58,413 - Début de l'entraînement du modèle (phase 1).
2025-01-29 15:01:58,421 - Traitement du batch 1/24: 32 images.
2025-01-29 15:02:10,530 - Batch 1 chargé : 32 images valides.


Epoch 1/30


KeyboardInterrupt: 

In [27]:
import os
import logging
import numpy as np
import pydicom
import tensorflow as tf
import cv2
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Configuration des logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# ====================
# PARAMÈTRES GÉNÉRAUX
# ====================
IMG_SIZE = 224
BATCH_SIZE = 16  # Reduced batch size for better generalization
EPOCHS_INITIAL = 40  # Increased epochs for better learning
EPOCHS_FINE = 20    # More fine-tuning epochs

# Chemin vers le fichier pickle
NOTEBOOK_DIRECTORY = "/Users/dtilm/Desktop/P1-Classification/notebook/balanced_data.pkl"

# ====================
# PREPROCESSING FUNCTIONS
# ====================
def enhance_mammogram(img):
    """
    Enhance mammogram image using CLAHE and percentile normalization
    """
    # Normalize to 0-255 range
    if img.dtype != np.uint8:
        p5 = np.percentile(img, 5)
        p95 = np.percentile(img, 95)
        img = np.clip(img, p5, p95)
        img = ((img - p5) / (p95 - p5) * 255).astype(np.uint8)
    
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(img)
    
    return enhanced

def preprocess_dicom(file_path):
    """
    Load and preprocess DICOM image with enhanced normalization and CLAHE
    """
    try:
        dicom = pydicom.dcmread(file_path, force=True)
        img = dicom.pixel_array.astype(np.float32)
        
        # Enhanced normalization
        img = enhance_mammogram(img)
        img = img.astype(np.float32) / 255.0
        
        # Convert to 3 channels
        img = np.stack([img, img, img], axis=-1)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        
        return img
    except Exception as e:
        logger.error(f"Erreur DICOM {file_path}: {e}")
        return None

# ====================
# DATA AUGMENTATION
# ====================
data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomFlip(mode="horizontal"),
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2),
])


def create_learning_rate_scheduler(warmup_epochs, initial_lr, total_epochs, steps_per_epoch):
    """
    Creates a learning rate schedule with warmup and cosine decay
    """
    warmup_steps = warmup_epochs * steps_per_epoch
    total_steps = total_epochs * steps_per_epoch
    
    def scheduler(epoch, lr):
        step = epoch * steps_per_epoch
        # Warmup phase
        if step < warmup_steps:
            return (step / warmup_steps) * initial_lr
        # Cosine decay phase
        else:
            progress = (step - warmup_steps) / (total_steps - warmup_steps)
            return initial_lr * 0.5 * (1 + np.cos(np.pi * progress))
    
    return tf.keras.callbacks.LearningRateScheduler(scheduler)

# ====================
# DATA GENERATOR
# ====================
class DICOMDataGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, augment=False, shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(len(self.file_paths))
            np.random.shuffle(indices)
            self.file_paths = [self.file_paths[i] for i in indices]
            self.labels = [self.labels[i] for i in indices]

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.file_paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        images = []
        valid_labels = []

        for path, lbl in zip(batch_paths, batch_labels):
            img = preprocess_dicom(path)
            if img is not None:
                if self.augment:
                    img = tf.expand_dims(img, 0)
                    img = data_augmentation(img)
                    img = tf.squeeze(img, axis=0)
                images.append(img)
                valid_labels.append(lbl)

        images = np.array(images)
        labels_array = np.array(valid_labels)

        return images, tf.keras.utils.to_categorical(labels_array, num_classes=3)

# ====================
# MODEL ARCHITECTURE
# ====================
def build_improved_model(img_size=224):
    """
    Creates an improved model based on EfficientNetB2 with custom head
    """
    base_model = EfficientNetB2(
        weights='imagenet',
        include_top=False,
        input_shape=(img_size, img_size, 3)
    )
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(3, activation='softmax')
    ])
    
    return model

# ====================
# FINE-TUNING HELPERS
# ====================
def apply_fine_tuning(model, unfreeze_percentage=0.3):
    """
    Applies fine-tuning strategy by unfreezing last X% of layers
    """
    base_model = model.layers[0]
    nb_layers = len(base_model.layers)
    
    base_model.trainable = True
    for layer in base_model.layers:
        layer.trainable = False
    
    for layer in base_model.layers[-int(nb_layers * unfreeze_percentage):]:
        layer.trainable = True
        
    return model

# ====================
# MAIN TRAINING SCRIPT
# ====================
def main():
    # Load and prepare data
    logger.info("Loading data...")
    balanced_data = pd.read_pickle(NOTEBOOK_DIRECTORY)
    
    # Filter data
    filtered_data = balanced_data[
        ['image_file_path_dicom', 'pathology', 'abnormality_type', 
         'left_or_right_breast', 'image_view']
    ].dropna()
    
    # Create labels
    label_mapping = {'BENIGN_WITHOUT_CALLBACK': 0, 'BENIGN': 1, 'MALIGNANT': 2}
    filtered_data['label'] = filtered_data['pathology'].map(label_mapping)
    
    # Split data
    train_data, val_data = train_test_split(
        filtered_data, 
        test_size=0.2, 
        random_state=42, 
        stratify=filtered_data['label']
    )
    
    # Create generators
    gen_train = DICOMDataGenerator(
        train_data['image_file_path_dicom'].tolist(),
        train_data['label'].tolist(),
        BATCH_SIZE,
        augment=True,
        shuffle=True
    )
    
    gen_val = DICOMDataGenerator(
        val_data['image_file_path_dicom'].tolist(),
        val_data['label'].tolist(),
        BATCH_SIZE,
        augment=False,
        shuffle=False
    )
    
    # Create callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            mode='max'
        ),
        ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.7,
            patience=5,
            verbose=1,
            mode='max'
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.keras',  # Changed from .h5 to .keras
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        )
    ]
    
    # Build and compile model
    logger.info("Building model...")
    model = build_improved_model(IMG_SIZE)
    
    # Initial training phase
    logger.info("Starting initial training phase...")
    initial_lr = 2e-4

    # Add LR scheduler to callbacks
    lr_scheduler = create_learning_rate_scheduler(
        warmup_epochs=5,
        initial_lr=initial_lr,
        total_epochs=EPOCHS_INITIAL,
        steps_per_epoch=len(gen_train)
    )
    callbacks.append(lr_scheduler)

    # Use constant initial learning rate for optimizer
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    history = model.fit(
        gen_train,
        validation_data=gen_val,
        epochs=EPOCHS_INITIAL,
        callbacks=callbacks,
        verbose=1
    )

    # Fine-tuning phase
    logger.info("Starting fine-tuning phase...")
    model = apply_fine_tuning(model)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    history_fine = model.fit(
        gen_train,
        validation_data=gen_val,
        epochs=EPOCHS_FINE,
        callbacks=callbacks,
        verbose=1
    )

    
    # Save final model
    logger.info("Saving final model...")
    model.save('final_model.keras')
    
    return history, history_fine

if __name__ == "__main__":
    main()

2025-01-29 13:38:58,565 - Loading data...


TypeError: issubclass() arg 1 must be a class

In [21]:
import os
import logging
import numpy as np
import pydicom
import tensorflow as tf
import cv2
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV3Small

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# ====================
# OPTIMIZED PARAMETERS
# ====================
IMG_SIZE = 224
BATCH_SIZE = 8
EPOCHS_INITIAL = 40
INITIAL_LR = 1e-4
WEIGHT_DECAY = 1e-3
WARMUP_EPOCHS = 2

# Chemin vers le fichier pickle
NOTEBOOK_DIRECTORY = "/Users/dtilm/Desktop/P1-Classification/notebook/balanced_data.pkl"

# ====================
# MEMORY MANAGEMENT
# ====================
def limit_memory_growth():
    try:
        tf.config.set_logical_device_configuration(
            tf.config.list_physical_devices('CPU')[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]
        )
    except:
        pass

# ====================
# LEARNING RATE SCHEDULER
# ====================
def cosine_decay_with_warmup(epoch, total_epochs, warmup_epochs=2):
    if epoch < warmup_epochs:
        return INITIAL_LR * ((epoch + 1) / warmup_epochs)
    
    progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
    return INITIAL_LR * (0.5 * (1 + np.cos(np.pi * progress)) + 0.1)

# ====================
# IMAGE PREPROCESSING
# ====================
def enhance_mammogram(img):
    p1, p99 = np.percentile(img, (1, 99))
    img = np.clip(img, p1, p99)
    img = ((img - p1) / (p99 - p1) * 255).astype(np.uint8)
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    
    return img

def preprocess_dicom(file_path):
    try:
        dicom = pydicom.dcmread(file_path, force=True)
        img = dicom.pixel_array.astype(np.float32)
        
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = enhance_mammogram(img)
        img = (img.astype(np.float32) - 127.5) / 127.5
        img = np.stack([img] * 3, axis=-1)
        
        return img
    except Exception as e:
        logger.error(f"Error processing DICOM {file_path}: {e}")
        return None

# ====================
# DATA AUGMENTATION
# ====================
def augment_image(img):
    if np.random.random() > 0.5:
        img = cv2.flip(img, 1)
    
    if np.random.random() > 0.5:
        angle = np.random.uniform(-10, 10)
        M = cv2.getRotationMatrix2D((IMG_SIZE//2, IMG_SIZE//2), angle, 1)
        img = cv2.warpAffine(img, M, (IMG_SIZE, IMG_SIZE))
    
    if np.random.random() > 0.5:
        img = img * np.random.uniform(0.8, 1.2)
        img = np.clip(img, -1, 1)
    
    return img

# ====================
# DATA GENERATOR
# ====================
class EnhancedGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, augment=False, shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.cache = {}
        self.indices = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        if len(self.cache) > 100:
            keys_to_remove = np.random.choice(list(self.cache.keys()), 
                                            size=len(self.cache)//2, 
                                            replace=False)
            for k in keys_to_remove:
                del self.cache[k]
    
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, len(self.file_paths))
        batch_indices = self.indices[start_idx:end_idx]
        
        images = []
        labels = []
        
        for i in batch_indices:
            path = self.file_paths[i]
            
            if path in self.cache:
                img = self.cache[path].copy()
            else:
                img = preprocess_dicom(path)
                if len(self.cache) < 100:
                    self.cache[path] = img.copy()
            
            if img is not None:
                if self.augment:
                    img = augment_image(img)
                images.append(img)
                labels.append(self.labels[i])
        
        if not images:
            return self.__getitem__((idx + 1) % self.__len__())
        
        return np.array(images), tf.keras.utils.to_categorical(labels, num_classes=3)

# ====================
# MODEL ARCHITECTURE
# ====================
def build_optimized_model(img_size=224):
    regularizer = tf.keras.regularizers.l2(WEIGHT_DECAY)
    
    base_model = MobileNetV3Small(
        include_top=False,
        weights='imagenet',
        input_shape=(img_size, img_size, 3),
        include_preprocessing=False
    )
    
    # Unfreeze more layers for better feature extraction
    for layer in base_model.layers[:-50]:
        layer.trainable = False
    
    inputs = layers.Input(shape=(img_size, img_size, 3))
    x = base_model(inputs)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    
    # Simplified architecture with stronger regularization
    x = layers.Dense(512, kernel_regularizer=regularizer, bias_regularizer=regularizer)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(256, kernel_regularizer=regularizer, bias_regularizer=regularizer)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.3)(x)
    
    outputs = layers.Dense(3, activation='softmax', kernel_regularizer=regularizer)(x)
    
    return tf.keras.Model(inputs, outputs)

# ====================
# TRAINING MONITOR
# ====================
class TrainingMonitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logger.info(
            f"Epoch {epoch + 1} - "
            f"train_acc: {logs.get('accuracy', 0):.4f}, "
            f"val_acc: {logs.get('val_accuracy', 0):.4f}, "
            f"train_loss: {logs.get('loss', 0):.4f}, "
            f"val_loss: {logs.get('val_loss', 0):.4f}"
        )

# ====================
# MAIN TRAINING SCRIPT
# ====================
def main():
    limit_memory_growth()
    
    logger.info("Loading data...")
    balanced_data = pd.read_pickle(NOTEBOOK_DIRECTORY)
    
    filtered_data = balanced_data[
        ['image_file_path_dicom', 'pathology', 'abnormality_type', 
         'left_or_right_breast', 'image_view']
    ].dropna()
    
    label_mapping = {'BENIGN_WITHOUT_CALLBACK': 0, 'BENIGN': 1, 'MALIGNANT': 2}
    filtered_data['label'] = filtered_data['pathology'].map(label_mapping)
    
    train_data, val_data = train_test_split(
        filtered_data, 
        test_size=0.2, 
        random_state=42, 
        stratify=filtered_data['label']
    )
    
    gen_train = EnhancedGenerator(
        train_data['image_file_path_dicom'].tolist(),
        train_data['label'].tolist(),
        BATCH_SIZE,
        augment=True,
        shuffle=True
    )
    
    gen_val = EnhancedGenerator(
        val_data['image_file_path_dicom'].tolist(),
        val_data['label'].tolist(),
        BATCH_SIZE,
        augment=False,
        shuffle=False
    )
    
    logger.info("Building model...")
    model = build_optimized_model(IMG_SIZE)
    
    # Use legacy optimizer for M1 Mac without weight_decay
    optimizer = tf.keras.optimizers.legacy.Adam(
        learning_rate=INITIAL_LR,
        clipnorm=1.0
    )
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            mode='max',
            min_delta=0.001
        ),
        ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.7,
            patience=5,
            verbose=1,
            mode='max',
            min_delta=0.001
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath='best_model.keras',
            monitor='val_accuracy',
            save_weights_only=True,
            mode='max'
        ),
        TrainingMonitor(),
        tf.keras.callbacks.LearningRateScheduler(
            lambda epoch: cosine_decay_with_warmup(epoch, EPOCHS_INITIAL)
        )
    ]
    
    logger.info("Starting training...")
    history = model.fit(
        gen_train,
        validation_data=gen_val,
        epochs=EPOCHS_INITIAL,
        callbacks=callbacks,
        verbose=1
    )
    
    logger.info("Saving model...")
    model.save('final_model.keras')
    
    return history

if __name__ == "__main__":
    main()

2025-01-29 16:14:19,044 - Loading data...
2025-01-29 16:14:19,062 - Building model...
2025-01-29 16:14:19,732 - Starting training...


Epoch 1/40
69/95 [====================>.........] - ETA: 43s - loss: 2.4230 - accuracy: 0.3382

KeyboardInterrupt: 

In [ ]:
import os
import logging
import numpy as np
import pydicom
import tensorflow as tf
import cv2
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
from tensorflow.keras.applications import MobileNetV3Small

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# ====================
# OPTIMIZED PARAMETERS
# ====================
IMG_SIZE = 224
BATCH_SIZE = 16  # Increased for better stability
EPOCHS_INITIAL = 40
INITIAL_LR = 1e-4
WEIGHT_DECAY = 1e-4

# File path
NOTEBOOK_DIRECTORY = "/Users/dtilm/Desktop/P1-Classification/notebook/balanced_data.pkl"

# ====================
# SAFE DATA LOADING
# ====================
def load_data_safely():
    """Safely load pickle data with error handling"""
    try:
        logger.info("Loading data...")
        return pd.read_pickle(NOTEBOOK_DIRECTORY)
    except Exception as e:
        logger.error(f"Error loading pickle: {e}")
        raise

# ====================
# IMAGE PREPROCESSING
# ====================
def enhance_mammogram(img):
    """Enhanced mammogram preprocessing"""
    p1, p99 = np.percentile(img, (1, 99))
    img = np.clip(img, p1, p99)
    img = ((img - p1) / (p99 - p1) * 255).astype(np.uint8)
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    
    return img

def preprocess_dicom(file_path):
    """Optimized DICOM preprocessing"""
    try:
        dicom = pydicom.dcmread(file_path, force=True)
        img = dicom.pixel_array.astype(np.float32)
        
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = enhance_mammogram(img)
        img = (img.astype(np.float32) - 127.5) / 127.5
        img = np.stack([img] * 3, axis=-1)
        
        return img
    except Exception as e:
        logger.error(f"Error processing DICOM {file_path}: {e}")
        return None

# ====================
# DATA AUGMENTATION
# ====================
def augment_image(img):
    """Optimized data augmentation"""
    if np.random.random() > 0.5:
        img = cv2.flip(img, 1)
    
    if np.random.random() > 0.5:
        angle = np.random.uniform(-10, 10)
        M = cv2.getRotationMatrix2D((IMG_SIZE//2, IMG_SIZE//2), angle, 1)
        img = cv2.warpAffine(img, M, (IMG_SIZE, IMG_SIZE))
    
    if np.random.random() > 0.5:
        img = img * np.random.uniform(0.8, 1.2)
        img = np.clip(img, -1, 1)
    
    return img

# ====================
# DATA GENERATOR
# ====================
class EnhancedGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, augment=False, shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.cache = {}
        self.indices = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        if len(self.cache) > 100:  # Clear cache periodically
            self.cache.clear()
    
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, len(self.file_paths))
        batch_indices = self.indices[start_idx:end_idx]
        
        images = []
        labels = []
        
        for i in batch_indices:
            path = self.file_paths[i]
            if path in self.cache:
                img = self.cache[path].copy()
            else:
                img = preprocess_dicom(path)
                if len(self.cache) < 100:  # Limit cache size
                    self.cache[path] = img.copy()
            
            if img is not None:
                if self.augment:
                    img = augment_image(img)
                images.append(img)
                labels.append(self.labels[i])
        
        if not images:
            return self.__getitem__((idx + 1) % self.__len__())
        
        return np.array(images), tf.keras.utils.to_categorical(labels, num_classes=3)

# ====================
# MODEL ARCHITECTURE
# ====================
def build_optimized_model(img_size=224):
    """Optimized model architecture"""
    regularizer = tf.keras.regularizers.l2(WEIGHT_DECAY)
    
    base_model = MobileNetV3Small(
        include_top=False,
        weights='imagenet',
        input_shape=(img_size, img_size, 3),
        include_preprocessing=False
    )
    
    # Freeze early layers
    for layer in base_model.layers[:-30]:  # Only train the last 30 layers
        layer.trainable = False
    
    inputs = layers.Input(shape=(img_size, img_size, 3))
    x = base_model(inputs)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    
    x = layers.Dense(256, kernel_regularizer=regularizer)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.4)(x)
    
    outputs = layers.Dense(3, activation='softmax')(x)
    
    return tf.keras.Model(inputs, outputs)

# ====================
# TRAINING MONITOR
# ====================
class TrainingMonitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logger.info(
            f"Epoch {epoch + 1} - "
            f"train_acc: {logs.get('accuracy', 0):.4f}, "
            f"val_acc: {logs.get('val_accuracy', 0):.4f}, "
            f"train_loss: {logs.get('loss', 0):.4f}, "
            f"val_loss: {logs.get('val_loss', 0):.4f}"
        )

# ====================
# MAIN TRAINING SCRIPT
# ====================
def main():
    # Load data
    balanced_data = load_data_safely()
    
    filtered_data = balanced_data[
        ['image_file_path_dicom', 'pathology', 'abnormality_type', 
         'left_or_right_breast', 'image_view']
    ].dropna()
    
    label_mapping = {'BENIGN_WITHOUT_CALLBACK': 0, 'BENIGN': 1, 'MALIGNANT': 2}
    filtered_data['label'] = filtered_data['pathology'].map(label_mapping)
    
    # Data split
    train_data, val_data = train_test_split(
        filtered_data, 
        test_size=0.2, 
        random_state=42, 
        stratify=filtered_data['label']
    )
    
    # Create generators
    gen_train = EnhancedGenerator(
        train_data['image_file_path_dicom'].tolist(),
        train_data['label'].tolist(),
        BATCH_SIZE,
        augment=True,
        shuffle=True
    )
    
    gen_val = EnhancedGenerator(
        val_data['image_file_path_dicom'].tolist(),
        val_data['label'].tolist(),
        BATCH_SIZE,
        augment=False,
        shuffle=False
    )
    
    # Build and compile model
    logger.info("Building model...")
    model = build_optimized_model(IMG_SIZE)
    
    optimizer = tf.keras.optimizers.legacy.Adam(
        learning_rate=INITIAL_LR,
        clipnorm=1.0
    )
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            mode='max',
            min_delta=0.001
        ),
        ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.7,
            patience=5,
            verbose=1,
            mode='max',
            min_delta=0.001
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath='best_model.keras',
            monitor='val_accuracy',
            save_weights_only=True,
            mode='max'
        ),
        TrainingMonitor()
    ]
    
    # Train
    logger.info("Starting training...")
    history = model.fit(
        gen_train,
        validation_data=gen_val,
        epochs=EPOCHS_INITIAL,
        callbacks=callbacks,
        verbose=1
    )
    
    # Save model
    logger.info("Saving model...")
    model.save('final_model.keras')
    
    return history

if __name__ == "__main__":
    main()

2025-01-29 17:32:48,535 - Loading data...
2025-01-29 17:32:48,581 - Building model...
2025-01-29 17:32:49,335 - Starting training...


Epoch 1/40
48/48 [==============================] - ETA: 0s - loss: 1.2112 - accuracy: 0.4723

2025-01-29 17:36:23,341 - Epoch 1 - train_acc: 0.4723, val_acc: 0.4526, train_loss: 1.2112, val_loss: 1.1518


48/48 [==============================] - 207s 4s/step - loss: 1.2112 - accuracy: 0.4723 - val_loss: 1.1518 - val_accuracy: 0.4526 - lr: 1.0000e-04
Epoch 2/40
48/48 [==============================] - ETA: 0s - loss: 0.9091 - accuracy: 0.5910

2025-01-29 17:38:42,784 - Epoch 2 - train_acc: 0.5910, val_acc: 0.4737, train_loss: 0.9091, val_loss: 1.1869


...

2025-01-29 19:07:05,969 - Epoch 39 - train_acc: 0.9565, val_acc: 0.9000, train_loss: 0.1511, val_loss: 0.5684


48/48 [==============================] - 145s 3s/step - loss: 0.1511 - accuracy: 0.9565 - val_loss: 0.5684 - val_accuracy: 0.9000 - lr: 1.0000e-04
Epoch 40/40
48/48 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.9710

2025-01-29 19:09:32,054 - Epoch 40 - train_acc: 0.9710, val_acc: 0.9105, train_loss: 0.1236, val_loss: 0.5611


48/48 [==============================] - 146s 3s/step - loss: 0.1236 - accuracy: 0.9710 - val_loss: 0.5611 - val_accuracy: 0.9105 - lr: 1.0000e-04


2025-01-29 19:09:32,060 - Saving model...
